In [8]:
%load_ext autoreload
%autoreload 2
import json
import torch
from tqdm import tqdm
import shutil
from conf.config import Config
from transformers import AutoTokenizer, AutoModelForCausalLM

from datasets import GameDataset, SokobanLMDataset, LMazeLMDataset
from evaluate import evaluate
from utils import get_run_name, save_train_state, load_train_state

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/home/gray/Projects/lm-pcg/evaluate.py:122: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  def main(args: Config):


In [11]:
class Args():
    model = "gpt2"
    game = "l_maze"
    data_source = "l_maze"
    chunk_size = 128
    learning_rate = 1e-4
    exp_name = ""
    seed = 1

    gen_len = 128
    gen_temp = 1.0
    gen_beams = 5
    gen_top_k = 50
    gen_top_p = 1.0
    gen_typical_p = 1.0
    num_eval_samples = 50
args = Args()

In [3]:
# Load the model from the checkpoint
run_name = get_run_name(args)
output_dir = f"./logs/{run_name}"

model, _, global_step = load_train_state(output_dir)

model_mapping = {"gpt2": "gpt2",
                     "codeparrot": "lvwerra/codeparrot",
                     "java-gpt2": "microsoft/CodeGPT-small-java-adaptedGPT2",
                     "incoder-1B": "facebook/incoder-1B",
                     "incoder-6B": "facebook/incoder-6B"}

# Instantiate the tokenizer based on the model's name
model_name = model_mapping[args.model]
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.add_special_tokens({"pad_token": "PAD",
                                "bos_token": "START"})

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Attempting to load checkpoint from ./logs/l_maze/gpt2/l_maze/chunk_size-128_lr-0.0001/seed-1...


In [4]:
# Create the LMaze dataset
dataset = LMazeLMDataset(tokenizer,
                         args.model,
                         chunk_size=args.chunk_size)

Loading tokens from cache at ./caches/gpt2_l_mazes_train_all_token_ids.npy...


In [12]:
prop_accurate, prop_novel, prop_playable = evaluate(model, device, tokenizer, dataset, args, verbose=False, render_dir=None)

print("GENERATION PARAMETERS:")
print(f"\tLength: {args.gen_len}")
print(f"\tTemperature: {args.gen_temp}")
print(f"\tTop-k: {args.gen_top_k}")
print(f"\tTop-p: {args.gen_top_p}")
print(f"\tTypical-p: {args.gen_typical_p}")
print(f"\tBeams: {args.gen_beams}")

print(f"\nProportion accurate: {prop_accurate}")
print(f"Proportion playable: {prop_playable}")
print(f"Proportion novel: {prop_novel}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


GENERATION PARAMETERS:
	Length: 128
	Temperature: 1.0
	Top-k: 50
	Top-p: 1.0
	Typical-p: 1.0
	Beams: 5

Proportion accurate: 1.0
Proportion playable: 1.0
Proportion novel: 1.0
